#### Install Requirements

In [1]:
%%capture
!pip install roboflow
import zipfile

import os, requests
from roboflow import Roboflow
rf = Roboflow(api_key="yX1nJyVNr91CZv1vcBAf")
project = rf.workspace("minemy").project("shoga-segmentation-combined-14030118")
version = project.version(2)
dataset = version.download("coco")

!pip install -q condacolab
import condacolab
condacolab.install()

In [2]:
%%capture
!git clone https://github.com/KKMOfficial/yolact-modified.git
%cd yolact-modified

In [2]:
%%capture
!activate yolact-env
!pip install opencv-python
!pip install pycocotools
!pip install torch torchvision torchaudio
!conda install -c conda-forge tensorboard
!pip install numpy==1.21.6
!pip install matplotlib==3.4.0
!pip install ipykernel
!pip install wget


In [1]:
#@title source code modifications
# line 131
# shoga_dataset = dataset_base.copy({
#   'name': 'Shoga_Dataset',
#   'train_images':'/content/shoga-segmentation-combined-14030118-2/train',
#   'train_info':'/content/shoga-segmentation-combined-14030118-2/train/_annotations.coco.json',
#   'valid_images': '/content/shoga-segmentation-combined-14030118-2/valid',
#   'valid_info':   '/content/shoga-segmentation-combined-14030118-2/valid/_annotations.coco.json',
#   'class_names': ('bottle'),
#   'label_map': { 1:  1 },
# })
# line 664
# yolact_resnet101_bottle_config = coco_base_config.copy({
#   'name': "bottle_detection",
#   'dataset': shoga_dataset,
#   'num_classes': len(shoga_dataset.class_names) + 1,
# })
# make only one class bottle in the dataset file
# backbone.py : 143: state_dict = torch.load(path, map_location="cuda:0")

In [3]:
#@title downlaod backbone networks
import os, requests, zipfile
import wget

def download_file(url, dest_dir, name):
  os.makedirs(dest_dir, exist_ok=True)
  wget.download(url,out = f"{dest_dir}{name}")



darknet_url = 'https://drive.usercontent.google.com/download?id=1tvqFPd4bJtakOlmn-uIA492g2qurRChj&export=download&authuser=0&confirm=t&uuid=ab9c794e-700e-4d43-9dbf-a6a188005cad&at=APZUnTXB4dxsThsKb_eEtGNW_j1F%3A1721465302564'
resnet_url  = "https://drive.usercontent.google.com/download?id=17Y431j4sagFpSReuPNoFcj9h7azDTZFf&export=download&authuser=0&confirm=t&uuid=91c570f0-0155-418c-8149-f68579cb7dba&at=APZUnTU7DDgtr1BiDTit8v7e1Wxn%3A1721478488916"
resnet50_url= "https://drive.usercontent.google.com/download?id=1Jy3yCdbatgXa5YYIdTCRrSV0S9V5g1rn&export=download&authuser=0"
dataset_url = "https://drive.usercontent.google.com/download?id=1PcJ8uwd6OGBG3w97zyH5KoxPeaSUd7k5&export=download&authuser=0&confirm=t&uuid=ce9e234a-2d81-4c33-b5f2-dc157128e6c7&at=APZUnTXbYqAFBbcNVZ6KlnBtVIY8:1721600907146"
dest_dir    = '/content/yolact-modified/weights/'
# file name : resnet101_reducedfc.pth
# file name 2 : darknet53.pth
# resnet50 : resnet50-19c8e357.pth

download_file(resnet50_url, dest_dir, "resnet50-19c8e357.pth")
download_file(resnet_url, dest_dir, "resnet101_reducedfc.pth")
download_file(darknet_url, dest_dir, "darknet53.pth")


#### Train Network

In [ ]:
# %load_ext tensorboard
%reload_ext tensorboard
%tensorboard --logdir runs --samples_per_plugin "images=10000"

In [6]:
%cd /content/yolact-modified
!rm -rf runs
!activate yolact-env
!python train.py --config=yolact_resnet50_bottle_config --batch_size=8

/content/yolact-modified

EnvironmentNameNotFound: Could not find conda environment: yolact-env
You can list all discoverable environments with `conda info --envs`.


/usr/local/lib/python3.10/site-packages/torch/__init__.py:955: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:432.)
  _C._set_default_tensor_type(t)
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
/usr/local/lib/python3.10/site-packages/torch/jit/_recursive.py:314: UserWarning: 'downsample_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/torch/jit/_recursive.py:314: UserWarning: 'pred_layers' was found in ScriptModule c

#### Check Model

In [13]:
import os, zipfile

def folder_to_zip(folder, zip_file):
    with zipfile.ZipFile(zip_file, 'w', zipfile.ZIP_DEFLATED) as zip_f:
        for root, _, files in os.walk(folder):
            for file in files:
                zip_f.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), folder))

folder_to_zip('/content/tests_results', '/content/tests_results.zip')
print('Folder converted to ZIP successfully.')

Folder converted to ZIP successfully.


In [12]:
!python eval.py --trained_model=/content/yolact-modified/weights/bottle_detection_12_364_interrupt.pth --config=yolact_resnet50_bottle_config --score_threshold=0.3 --top_k=15 --images=/content/tests:/content/tests_results

/usr/local/lib/python3.10/site-packages/torch/__init__.py:955: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:432.)
  _C._set_default_tensor_type(t)
/usr/local/lib/python3.10/site-packages/torch/jit/_recursive.py:314: UserWarning: 'downsample_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/torch/jit/_recursive.py:314: UserWarning: 'pred_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/torch/jit/_recursive.py:314: UserWarning: 'lat_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  warnings.warn(
/content/yolact-modified/yolact.py:479:

In [11]:
import zipfile

def unzip(zip_file, extract_to):
    with zipfile.ZipFile(zip_file) as zip_f:
        zip_f.extractall(extract_to)

unzip('/content/tests.zip', '/content/tests')
print('File extracted successfully.')

File extracted successfully.
